In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import Adam


df = pd.read_excel('C:/Users/ruben/OneDrive - Pontificia Universidad Javeriana/Archivos de chat de Microsoft Teams/InstaCart 1.xlsx', engine='openpyxl')


# Supongamos que 'df' es tu DataFrame que contiene los datos de los pedidos

# Convertir los IDs de producto en enteros consecutivos
product_encoder = LabelEncoder()
df['product_id'] = product_encoder.fit_transform(df['product_id'])

# Agrupar por usuario y orden para crear secuencias
user_orders = df.sort_values(['user_id', 'order_id', 'add_to_cart_order']).groupby(['user_id', 'order_id'])['product_id'].apply(list)

# Crear secuencias de entrada y etiquetas (puedes definir el tamaño de la secuencia que desees)
X = []
y = []
sequence_length = 3  # Define la longitud de la secuencia que quieras usar

for order_sequence in user_orders:
    if len(order_sequence) > sequence_length:
        for i in range(len(order_sequence) - sequence_length):
            X.append(order_sequence[i:i+sequence_length])
            y.append(order_sequence[i+sequence_length])

# Convertir a arrays de numpy para el entrenamiento
X = np.array(X)
y = np.array(y)


from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train y y_train serán utilizados para entrenar el modelo
# X_val y y_val serán utilizados para validar el modelo

# Definir los parámetros de entrada
input_length = sequence_length  # La longitud de las secuencias de entrada
num_unique_products = len(product_encoder.classes_)  # Número de productos únicos


# Crear una matriz de interacciones usuario-ítem
interactions = df.pivot_table(index='user_id', columns='product_id', values='reordered', fill_value=0)


# Realizar la descomposición SVD
U, S, VT = np.linalg.svd(interactions, full_matrices=False)

# Seleccionar el número de componentes latentes (k)
k = 50
U_k = U[:, :k]
S_k = np.diag(S[:k])
VT_k = VT[:k, :]

# Reconstruir la matriz reducida
interactions_reduced = np.dot(np.dot(U_k, S_k), VT_k)

# Definir los parámetros de entrada
sequence_length = 3  # Ajusta según tus necesidades
num_unique_products = interactions_reduced.shape[1]  # Número de productos únicos

# Crear secuencias de entrenamiento
X = []
y = []

for i in range(interactions_reduced.shape[0]):
    for j in range(interactions_reduced.shape[1] - sequence_length):
        X.append(interactions_reduced[i, j:j + sequence_length])
        y.append(interactions_reduced[i, j + sequence_length])

X = np.array(X)
y = np.array(y)

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Redimensionar los datos para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]))  # Eliminar la dimensión adicional
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1]))  # Eliminar la dimensión adicional


# Iniciar el modelo
model = Sequential()

# Añadir una capa de Embedding
model.add(Embedding(input_dim=num_unique_products + 1, output_dim=50))

# Añadir capas LSTM
model.add(LSTM(units=100, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))

# Añadir capas densas
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(0.2))

# Añadir la capa de salida
model.add(Dense(num_unique_products, activation='softmax'))

# Definir el optimizador
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

# Compilar el modelo
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ver resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)



c:\Users\ruben\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
96057/96058 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1380 - loss: 0.1848

KeyboardInterrupt: 

In [ ]:
#          PREDICCIÓN

# Suponiendo que tienes un product_id específico que quieres probar
# Primero, debes transformarlo con el LabelEncoder para obtener su representación codificada
one = 5876 # Cambia esto por un product_id real de tu conjunto de datos
one = product_encoder.transform([one])
two = 49683  # Cambia esto por un product_id real de tu conjunto de datos
two = product_encoder.transform([two])
three = 26209  # Cambia esto por un product_id real de tu conjunto de datos
three = product_encoder.transform([three])

input_sequence = np.array([[one, two, three]]) 

# Hacer la predicción con el modelo
predicted_probabilities = model.predict(input_sequence)

# El modelo devuelve un conjunto de probabilidades para cada producto único,
# donde cada probabilidad corresponde a la probabilidad de que ese producto sea el siguiente.
# Para obtener la recomendación final, seleccionamos el ID del producto con la mayor probabilidad.
predicted_product_id_encoded = np.argmax(predicted_probabilities)

# Finalmente, debemos transformar este ID codificado de vuelta a su valor original
predicted_product_id = product_encoder.inverse_transform([predicted_product_id_encoded])

print(f'El siguiente producto recomendado es: {predicted_product_id[0]}')